In [2]:
from datasets import load_dataset
from setfit import SetFitModel
import pandas as pd

from tqdm.auto import tqdm
tqdm.pandas()

In [3]:
breach_model  = SetFitModel._from_pretrained('./classify/setfit/model_breach')
company_model = SetFitModel._from_pretrained('./classify/setfit/model_company')

In [6]:
# Run model on headlines!
DATA_PATH = './classify/irr/test_set_gpt/decilm.csv' #"./data/headlines.csv"
df = pd.read_csv(DATA_PATH).dropna(subset=['Headline'])

pred_breach  = breach_model.predict(df['Headline'].reset_index(drop=True)) # on cpu :(
pred_company = company_model.predict(df['Headline'].reset_index(drop=True)) # on cpu :(

In [12]:
sims  = 0
diffs = 0
b, c = 0, 0
for x, y in zip(pred_breach, pred_company):
    if x == y: sims += 1
    else:      diffs += 1
    if x and not y: b += 1
    if y and not x: c += 1

print(f"Columns labelled Breach AND Company: {sims}")
print(f"Columns labelled Breach XOR Company: {diffs}")
print(f"Columns labelled Breach and NOT Company: {b}")
print(f"Columns labelled NOT Company and Breach: {c}")

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0])

In [7]:
# Save predictions to a csv file
preds = pd.DataFrame({'BreachMentioned': [bool(p) for p in pred_breach], 'CompanyMenioned': [bool(p) for p in pred_company]})
df = pd.concat([df, preds], axis=1)
df

,Date,Publication,Headline,BreachMentioned,CompanyMentioned,BreachMentioned,CompanyMenioned
0,20210405,The Guardian,"India National record 103,558 new Covid cases ...",False,False,False,False
1,20130601,Washington Post,What the media forget about leaks,False,False,False,False
2,20080826,New York Times,Bits: AT&T Launches New Data Plans for iPhone ...,False,AT&T,False,False
3,20140316,New York Times,Pilot Spoke to Air Controllers After Shutdown ...,False,False,False,False
4,20160330,CNBC,Illinois' epic budget deadlock sets dubious r...,False,False,False,False
...,...,...,...,...,...,...,...
195,20160802,New York Times,3 Top D.N.C. Officials Leave in Wake of Email ...,True,Government,True,True
196,20180407,FOX,7-year-old girl hikes Mount Kilimanjaro in hon...,False,False,False,False
197,20160817,Washington Post,AT&T’s new data plans trade one overage penalt...,False,AT&T,False,False
198,20140531,Daily Mail,Jaques Costeau grandson aims to break his fame...,False,False,False,False


In [8]:
df.to_csv("./data/setfit_predictions.csv", index=False)